## 知识图谱构建
### 下面我们将根据一份非结构化数据，构建一个知识图谱
```
一般而言，我们拿到的数据是非结构化的，可能来自业务部门，也可能来自网络爬虫，针对用非结构化数据如何结构化，并导入到图数据库里，首选的技术栈是关系抽取和实体抽取，以及三元组的联合抽取，但这部分技术栈不是本次课程的重点，因此我们在构建图谱的时候，直接给出了已经结构化的数据。
```

#### 知识图谱schema
根据上面样本数据的特点，我们设计图谱如下：
* Disease（疾病）

|  属性   | 含义  |  例子  |
|  ----  | ----  | ----  |
| name  | 疾病名称 | "百日咳" |
| intro  | 简介 | "百日咳(pertussis，whoopingcough)是…" |
| cause  | 起因 | " (一)发病原因\n病原菌是鲍特菌属(Bordetella)…" |
| prevent  | 预防 | "百日咳预防\n1、控制传染源…" |
| nursing  | 护理 | "确诊的患者应立即隔离至病后40天…" |
| insurance  | 是否医保 | "否" |
| easy_get  | 易感人群 | "多见于小儿" |
| get_way  | 感染途径 | "呼吸道传播" |
| get_prob  | 感染概率 | "0.5%" |
| treat  | 治疗方式 | "药物治疗", "支持性治疗" |
| treat_prob  | 治愈概率 | "98%" |
| treat_period  | 治疗时间 | "1-2个月" |
| treat_cost  | 疾病名称 | "根据不同医院，收费标准不一致，市三甲…" |
| treat_detail  | 治疗细节 | "百日咳西医治疗 1、药物治疗…" |


* Department（科室）

|  属性   | 含义  |  例子  |
|  ----  | ----  | ----  |
| name  | 科室名称 | "小儿内科" |


* Drug（药物）

|  属性   | 含义  |  例子  |
|  ----  | ----  | ----  |
| name  | 药物名称 | "阿司匹林肠溶片" |


* Food（食物）

|  属性   | 含义  |  例子  |
|  ----  | ----  | ----  |
| name  | 食物名称 | "小米面" |

* Symptom（症状）

|  属性   | 含义  |  例子  |
|  ----  | ----  | ----  |
| name  | 症状名称 | "指甲呈筒形" |

* 关系设计（Relationship Design）：共有6种关系。

|  属性   | 含义  |  例子  |
|  ----  | ----  | ----  |
| has_symptom  | 有症状 | ("大叶性肺炎", "有症状", "高热")  |
| can_use_drug  | 可用药 | ("大叶性肺炎", "可以用药", "乳酸左氧氟沙星片") |
| belongs_to  | 属于 | ("大叶性肺炎", "属于", "呼吸内科") |
| has_neopathy  | 有并发症 | ("大叶性肺炎", "有并发症", "肺脓肿") |
| can_eat  | 可以吃 | ("大叶性肺炎", "可以吃", "鲫鱼") |
| not_eat  | 不能吃 | ("大叶性肺炎", "不能吃", "韭菜") |



In [9]:
# 构建图谱
!python process_data/process.py process_data

create disease node count=50
create disease node count=100
create disease node count=150
create disease node count=200
create disease node count=250
create disease node count=300
create disease node count=350
create disease node count=400
create disease node count=450
create disease node count=500
create disease node count=550
create disease node count=600
create disease node count=650
create disease node count=700
create disease node count=750
create disease node count=800
create disease node count=850
create disease node count=900
create disease node count=950
create disease node count=1000
create disease node count=1050
create disease node count=1100
create disease node count=1150
create disease node count=1200
create disease node count=1250
create disease node count=1300
create disease node count=1350
create disease node count=1400
create disease node count=1450
create disease node count=1500
create disease node count=1550
create disease node count=1600
create disease node count=16

### 图谱效果
查询语句：
> MATCH p=()-[:can_use_drug]->() RETURN p LIMIT 25

![](./img/can_use_drug.png)


#### 拿到我们这样一个图谱之后，怎么构建KBQA呢？
* 如果我们通过NLU做句子分类，发现当前用户输入时查疾病-症状的关系，并且用户输入的句子包含了疾病实体：
 ```json
大叶性肺炎有哪些症状哦
 ```
上面的问句，从意图来看，属于查询：疾病-症状
从实体提取看，能识别到疾病：大叶性肺炎
因此为了回答上面的问题，我们可以将其转为针对我们图谱的一个查询，这样就完整了我们KBQA的一个最简单的建模过程啦。
###### 对应的查询语句为：
> MATCH (p:Disease{name: "大叶性肺炎"})-[:has_symptom]->(s:Symptom) return s.name

![](./img/has_symptom.png)